<a href="https://colab.research.google.com/github/rahulkhn/MarketDataAPI/blob/master/TraderMadeForexAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from pprint import PrettyPrinter
import numpy as np
import pandas as pd
pp = PrettyPrinter()

In [ ]:
url = "https://marketdata.tradermade.com/api/v1/live"

currency = "USDJPY,GBPUSD,UK100"
api_key = "api_key"
querystring = {"currency":currency,"api_key":api_key}

response = requests.get(url, params=querystring)

pp.pprint(response.json())

{'endpoint': 'live',
 'quotes': [{'ask': 108.774,
             'base_currency': 'USD',
             'bid': 108.773,
             'mid': 108.7735,
             'quote_currency': 'JPY'},
            {'ask': 1.38048,
             'base_currency': 'GBP',
             'bid': 1.38046,
             'mid': 1.38047,
             'quote_currency': 'USD'},
            {'ask': 6709.4,
             'bid': 6708.1,
             'instrument': 'UK100',
             'mid': 6708.75}],
 'requested_time': 'Tue, 23 Mar 2021 09:12:55 GMT',
 'timestamp': 1616490776}


In [ ]:
import pandas as pd
df = pd.DataFrame(response.json()["quotes"])
df

,ask,base_currency,bid,mid,quote_currency,instrument
0,108.78200,USD,108.78200,108.78200,JPY,NaN
1,1.38657,GBP,1.38655,1.38656,USD,NaN
2,6717.10000,NaN,6715.70000,6716.39990,NaN,UK100


In [ ]:
df = pd.DataFrame(response.json()["quotes"])
df["instrument"] = np.where(df["base_currency"].isnull(),df["instrument"],
                            df["base_currency"]+df["quote_currency"])
df["timestamp"] = response.json()["timestamp"]
df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
df = df[["instrument","bid","mid","ask","timestamp"]]
df

,instrument,bid,mid,ask,timestamp
0,USDJPY,108.76200,108.76250,108.76300,2021-03-22 17:42:59
1,GBPUSD,1.38678,1.38678,1.38678,2021-03-22 17:42:59
2,UK100,6721.20000,6721.89990,6722.60000,2021-03-22 17:42:59


In [ ]:
df.T

,0,1,2
instrument,USDJPY,GBPUSD,UK100
bid,108.697,1.38638,6722.6
mid,108.698,1.38639,6723.35
ask,108.698,1.3864,6724.1
timestamp,2021-03-22 16:29:11,2021-03-22 16:29:11,2021-03-22 16:29:11


In [ ]:
df.to_csv('live_rates.csv')

In [ ]:
url = "https://marketdata.tradermade.com/api/v1/historical"
date = "2021-03-15"
querystring = {"currency":currency,"date":date, "api_key":api_key}
response = requests.get(url, params=querystring)

df = pd.DataFrame(response.json()["quotes"])
df["instrument"] = np.where(df["base_currency"].isnull(),df["instrument"],
                            df["base_currency"]+df["quote_currency"])
df["date"] = response.json()["date"]
df["date"] = pd.to_datetime(df["date"])
df = df[["instrument","open","high","low", "close","date"]]
df

,instrument,open,high,low,close,date
0,USDJPY,109.01950,109.36700,108.91500,109.15650,2021-03-15
1,GBPUSD,1.39208,1.39501,1.38534,1.39003,2021-03-15
2,UK100,6786.10010,6798.29980,6716.70020,6764.29980,2021-03-15


In [ ]:
fx = ["EURUSD", "USDJPY"]
dates = ["2021-03-15-13:00"]
array = []
url = "https://marketdata.tradermade.com/api/v1/minute_historical"

for i in fx:
    for date in dates: 
        querystring = {"currency":i,"date_time":date, "api_key":api_key}
        response = requests.get(url, params=querystring)
        array.append(response.json())
df = pd.DataFrame(array)
df

,close,currency,date_time,endpoint,high,low,open,request_time
0,1.19241,EURUSD,2021-03-15-13:00,minute_historical,1.19256,1.19234,1.19248,"Mon, 22 Mar 2021 18:06:37 GMT"
1,109.16900,USDJPY,2021-03-15-13:00,minute_historical,109.17350,109.15700,109.16000,"Mon, 22 Mar 2021 18:06:38 GMT"


In [ ]:
url = "https://marketdata.tradermade.com/api/v1/timeseries?"
currency="USDJPY"
start_date="2021-03-17-13:00"
end_date="2021-03-22-13:00"
format="split"
interval="hourly"

df =pd.read_json('https://marketdata.tradermade.com/api/v1/timeseries?currency='
                +currency+'&api_key='+api_key+'&start_date='+start_date+'&end_date='
                +end_date+'&format='+format+'&interval='+interval)
df = pd.DataFrame(df.quotes['data'], columns=df.quotes['columns'])
df.tail()

,date,open,high,low,close
68,2021-03-22 09:00:00,108.6740,108.6925,108.6205,108.6530
69,2021-03-22 10:00:00,108.6545,108.7885,108.6335,108.7400
70,2021-03-22 11:00:00,108.7390,108.7450,108.6490,108.7005
71,2021-03-22 12:00:00,108.7020,108.8615,108.6890,108.8260
72,2021-03-22 13:00:00,108.8250,108.8460,108.7420,108.7705


In [ ]:
currency="EURUSD"
start_date="2021-03-22%2014:00"
end_date="2021-03-22%2014:30"
format="csv"

url = "https://marketdata.tradermade.com/api/v1/tick_historical_sample/"+currency+"/"+start_date+"/"+end_date+"?api_key="+api_key+"&format="+format

df =pd.read_csv(url)

df.tail()

,time,inst,bid,ask
1983,2021-03-22 14:29:57.530000+00:00,EURUSD,1.19177,1.1917
1984,2021-03-22 14:29:57.829000+00:00,EURUSD,1.19178,1.1917
1985,2021-03-22 14:29:58.131000+00:00,EURUSD,1.19179,1.1917
1986,2021-03-22 14:29:59.623000+00:00,EURUSD,1.19178,1.1917
1987,2021-03-22 14:29:59.928000+00:00,EURUSD,1.19179,1.1917
